# EDA

In [103]:
!pip install openpyxl

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [72]:
import os
os.chdir('/Users/ayushyajnik/Desktop/Python/lapse_rate_app/')

In [70]:
%pwd

'/Users/ayushyajnik/Desktop/Python'

In [73]:
df = pd.read_csv("artifacts/data_ingestion/lapse-rate.csv")


In [106]:
df

,CHANNEL1,CHANNEL2,CHANNEL3,ENTRY AGE,SEX,POLICY TYPE 1,POLICY TYPE 2,POLICY TYPE 3,PAYMENT MODE,POLICY STATUS,BENEFIT,NON LAPSE GUARANTEED,SUBSTANDARD RISK,NUMBER OF ADVANCE PREMIUM,INITIAL BENEFIT,Full Benefit?,Policy Year (Decimal),Policy Year,Premium,Issue Date
0,1,1,1,36,F,1,1,A,Annually,Inforce,200000.0,NO NLG,0.0,0,0.0,N,8.000000,9,280.0,11-Aug
1,1,1,1,42,M,1,2,A,Annually,Inforce,100000.0,NO NLG,0.0,0,0.0,N,8.000000,9,200.0,11-Aug
2,1,1,2,40,M,2,3,A,Annually,Lapse,80000.0,NO NLG,0.0,0,0.0,N,8.000000,9,1289.0,11-Aug
3,1,1,3,39,M,1,2,A,Monthly,Lapse,100000.0,NO NLG,0.0,0,0.0,N,8.000000,9,216.0,11-Aug
4,1,1,4,44,M,1,2,A,Annually,Lapse,100000.0,NO NLG,0.0,0,0.0,N,7.916667,8,140.0,11-Sep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185555,8,3,0,31,F,12,58,A,Annually,Inforce,247158.0,NO NLG,0.0,0,0.0,N,0.000000,1,5000.0,19-Aug
185556,8,3,0,36,M,12,58,A,Annually,Inforce,200000.0,NO NLG,0.0,0,0.0,N,0.000000,1,4734.0,19-Aug
185557,8,3,0,24,M,8,52,C,Quaterly,Inforce,17000.0,NO NLG,0.0,0,0.0,N,0.000000,1,NaN,19-Aug
185558,8,3,0,35,F,12,58,A,Annually,Inforce,100000.0,NO NLG,0.0,0,0.0,N,0.000000,1,2023.0,19-Aug


In [107]:
df['Issue Date'].value_counts()

Issue Date
16-Dec    4884
16-Jun    4319
15-Dec    3739
16-Nov    3731
15-Jun    3417
          ... 
11-Nov      86
11-Oct      82
11-Sep      79
11-Aug      76
12-Jan      70
Name: count, Length: 97, dtype: int64

In [108]:
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, chi2

# 1. Select columns with object dtype
object_columns = df.select_dtypes(include=['object']).columns

df_non_categorical = df.drop(columns=object_columns)

# 2. Apply OrdinalEncoder to object columns
encoder = OrdinalEncoder()
df_encoded = pd.DataFrame(encoder.fit_transform(df[object_columns]), 
                          columns=object_columns, 
                          index=df.index)

In [110]:
df_encoded

,SEX,POLICY TYPE 3,PAYMENT MODE,POLICY STATUS,NON LAPSE GUARANTEED,Full Benefit?,Issue Date
0,0.0,0.0,0.0,2.0,3.0,0.0,0.0
1,1.0,0.0,0.0,2.0,3.0,0.0,0.0
2,1.0,0.0,0.0,3.0,3.0,0.0,0.0
3,1.0,0.0,1.0,3.0,3.0,0.0,0.0
4,1.0,0.0,0.0,3.0,3.0,0.0,4.0
...,...,...,...,...,...,...,...
185555,0.0,0.0,0.0,2.0,3.0,0.0,90.0
185556,1.0,0.0,0.0,2.0,3.0,0.0,90.0
185557,1.0,2.0,2.0,2.0,3.0,0.0,90.0
185558,0.0,0.0,0.0,2.0,3.0,0.0,90.0


In [111]:
# 3. Apply chi-squared test
# Assuming 'target' is your target variable. Replace it with your actual target column name
target = df_encoded['POLICY STATUS']  # Replace 'target' with your actual target column name
selector = SelectKBest(chi2, k='all')  # Select all features initially
selector.fit(df_encoded, target)

# Get feature scores and p-values
scores = pd.DataFrame({
    'feature': object_columns,
    'score': selector.scores_,
    'p_value': selector.pvalues_
})

# Sort features by score in descending order
scores = scores.sort_values('score', ascending=False)

In [112]:
print(scores)

                feature          score       p_value
6            Issue Date  212279.617087  0.000000e+00
3         POLICY STATUS   28820.519036  0.000000e+00
1         POLICY TYPE 3    4095.632968  0.000000e+00
2          PAYMENT MODE     403.080673  6.006970e-86
5         Full Benefit?     122.214141  1.797535e-25
0                   SEX      33.356863  1.009469e-06
4  NON LAPSE GUARANTEED       4.959311  2.914980e-01


In [113]:
# 4. Select top features (e.g., top 5 or based on p-value)
top_features = scores.head(5)['feature'].tolist()  # Select top 5 features
# Alternatively, you can select based on p-value:
# top_features = scores[scores['p_value'] < 0.05]['feature'].tolist()


In [114]:
print(top_features)

['Issue Date', 'POLICY STATUS', 'POLICY TYPE 3', 'PAYMENT MODE', 'Full Benefit?']


In [115]:
# 5. Create DataFrame with selected features
df_selected = df_encoded[top_features]

In [116]:
df_selected

,Issue Date,POLICY STATUS,POLICY TYPE 3,PAYMENT MODE,Full Benefit?
0,0.0,2.0,0.0,0.0,0.0
1,0.0,2.0,0.0,0.0,0.0
2,0.0,3.0,0.0,0.0,0.0
3,0.0,3.0,0.0,1.0,0.0
4,4.0,3.0,0.0,0.0,0.0
...,...,...,...,...,...
185555,90.0,2.0,0.0,0.0,0.0
185556,90.0,2.0,0.0,0.0,0.0
185557,90.0,2.0,2.0,2.0,0.0
185558,90.0,2.0,0.0,0.0,0.0


In [117]:

# 7. Combine remaining original DataFrame with selected encoded features
df_final = pd.concat([df_non_categorical, df_selected], axis=1)

In [118]:
# Print results
print("Original DataFrame shape:", df.shape)
print("Final DataFrame shape:", df_final.shape)
print("Selected features:", top_features)
print("\nFeature scores:")
print(scores)


Original DataFrame shape: (185560, 20)
Final DataFrame shape: (185560, 18)
Selected features: ['Issue Date', 'POLICY STATUS', 'POLICY TYPE 3', 'PAYMENT MODE', 'Full Benefit?']

Feature scores:
                feature          score       p_value
6            Issue Date  212279.617087  0.000000e+00
3         POLICY STATUS   28820.519036  0.000000e+00
1         POLICY TYPE 3    4095.632968  0.000000e+00
2          PAYMENT MODE     403.080673  6.006970e-86
5         Full Benefit?     122.214141  1.797535e-25
0                   SEX      33.356863  1.009469e-06
4  NON LAPSE GUARANTEED       4.959311  2.914980e-01


In [119]:
df_final.head()

,CHANNEL1,CHANNEL2,CHANNEL3,ENTRY AGE,POLICY TYPE 1,POLICY TYPE 2,BENEFIT,SUBSTANDARD RISK,NUMBER OF ADVANCE PREMIUM,INITIAL BENEFIT,Policy Year (Decimal),Policy Year,Premium,Issue Date,POLICY STATUS,POLICY TYPE 3,PAYMENT MODE,Full Benefit?
0,1,1,1,36,1,1,200000.0,0.0,0,0.0,8.000000,9,280.0,0.0,2.0,0.0,0.0,0.0
1,1,1,1,42,1,2,100000.0,0.0,0,0.0,8.000000,9,200.0,0.0,2.0,0.0,0.0,0.0
2,1,1,2,40,2,3,80000.0,0.0,0,0.0,8.000000,9,1289.0,0.0,3.0,0.0,0.0,0.0
3,1,1,3,39,1,2,100000.0,0.0,0,0.0,8.000000,9,216.0,0.0,3.0,0.0,1.0,0.0
4,1,1,4,44,1,2,100000.0,0.0,0,0.0,7.916667,8,140.0,4.0,3.0,0.0,0.0,0.0


In [120]:
target.value_counts()

POLICY STATUS
3.0    98865
2.0    65814
4.0    20233
0.0      513
1.0      135
Name: count, dtype: int64

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    data:Path
    root_dir: Path
    transformed_data_path: Path
    trainData: Path
    testData: Path
    training_independent_data: Path
    training_dependent_data: Path
    testing_independent_data: Path
    testing_dependent_data: Path
    

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/Users/ayushyajnik/Desktop/Python/lapse_rate_app'

In [7]:
!pip install python-box


In [9]:
!pip install ensure


In [5]:
from src.No_More_Lapses.constants import *
from src.No_More_Lapses.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])
        create_directories([config.trainData])
        create_directories([config.testData])

        data_transformation_config = DataTransformationConfig(
            data = config.original_data,
            root_dir=config.root_dir,
            transformed_data_path = config.transformed_data_path,
            trainData = config.trainData,
            testData = config.testData,
            training_independent_data=config.training_independent_data,
            training_dependent_data=config.training_dependent_data,
            testing_independent_data= config.testing_independent_data,
            testing_dependent_data= config.testing_dependent_data
        )

        return data_transformation_config

In [7]:
a = ConfigurationManager()

[2025-03-23 18:55:12,797: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-23 18:55:12,799: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-23 18:55:12,800: INFO: common: created directory at: artifacts]


In [109]:
a.get_data_transformation_config()

[2025-03-23 16:58:17,520: INFO: common: created directory at: artifacts/transformed_data]
[2025-03-23 16:58:17,522: INFO: common: created directory at: artifacts/transformed_data/trainining_data]
[2025-03-23 16:58:17,523: INFO: common: created directory at: artifacts/transformed_data/testing_data]


DataTransformationConfig(data='artifacts/data_ingestion/lapse-rate.csv', root_dir='artifacts/transformed_data', transformed_data_path='artifacts/transformed_data/lapsed_data_encoded.csv', trainData='artifacts/transformed_data/trainining_data', testData='artifacts/transformed_data/testing_data', training_independent_data='artifacts/transformed_data/trainining_data/X_train.csv', training_dependent_data='artifacts/transformed_data/trainining_data/y_train.csv', testing_independent_data='artifacts/transformed_data/testing_data/X_test.csv', testing_dependent_data='artifacts/transformed_data/testing_data/y_test.csv')

In [8]:
import os
from src.No_More_Lapses import logger

In [15]:
!pip install --upgrade scikit-learn==1.3.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 30.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [16]:
!pip install --upgrade scikit-learn imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 23.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0


In [13]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.impute import SimpleImputer



class DataTransformer:
    
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def encode_object_columns(self):

        global df_non_categorical, object_columns
        
        # 1. Select columns with object dtype
        logger.info("Reading the original dataset for extracting object data type")
        dataframe = pd.read_csv(self.config.data)
        logger.info("Extracting columns with object data type")
        object_columns = dataframe.select_dtypes(include=['object']).columns
        logger.info("creating a dataframe with columns with no object data type columns")
        df_non_categorical = dataframe.drop(columns=object_columns)

        logger.info("Creating Ordinal encoder object")
        encoder = OrdinalEncoder()
        df_encoded = pd.DataFrame(encoder.fit_transform(dataframe[object_columns]), 
                                columns=object_columns, 
                                index=dataframe.index)
        logger.info("Encoded the object columns and created a dataframe from it.")
        
        return df_encoded
    
    def extract_target(self):

        encoded_df = self.encode_object_columns()
        target = encoded_df['POLICY STATUS']
        logger.info("Creating a variable with encoded target variable.")
        return target
    
    def apply_chi_squared_test(self):

        global dependent_var
        encoded_data = self.encode_object_columns()
        dependent_var = self.extract_target()

        selector = SelectKBest(chi2, k='all')  # Select all features initially
        selector.fit(encoded_data, dependent_var)
        logger.info("Applied Chi-Squared test to get the most important features")

        # Get feature scores and p-values
        scores = pd.DataFrame({
            'feature': object_columns,
            'score': selector.scores_,
            'p_value': selector.pvalues_
        })
        top_features = scores.head(5)['feature'].tolist()

        # Sort features by score in descending order
        scores = scores.sort_values('score', ascending=False)
        scores.to_csv(self.config.root_dir+'/scores.csv')
        logger.info("Exported Chi-squared test to data transformation root directory.")

        df_selected = encoded_data[top_features]
        logger.info("Created a separate dataframe with encoded important features")

        return df_selected
    
    def final_dataframe(self):

        cat_encoded_df = self.apply_chi_squared_test()

        logger.info("Concatenating the encoded categorical columns with numerical/float columns")

        df_final = pd.concat([df_non_categorical, cat_encoded_df], axis=1)
        df_final.to_csv(self.config.transformed_data_path)
        logger.info("Exported the encded data for archival purposes.")
        return df_final
    
    def train_test_split(self):
        
        imputer = SimpleImputer(strategy='median')

        final_encoded_dataframe = self.final_dataframe()
        # Drop 'POLICY STATUS' and 'Unnamed: 0' from the features
        X = final_encoded_dataframe.drop(['POLICY STATUS', 'Unnamed: 0'], axis=1, errors='ignore')

        X_train, X_test, y_train, y_test = train_test_split(X, dependent_var, test_size=0.2, random_state=42)
        logger.info("Data has been bifurcated into training and testing dataset")

        X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
        logger.info("SimpleImputer has been applied to the training data")


        # Apply SMOTE to the training data
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_imputed, y_train)
        logger.info("SMOTE has been applied for class imabalnce problem.")

        X_train_resampled.to_csv(self.config.training_independent_data)
        y_train_resampled.to_csv(self.config.training_dependent_data)

        logger.info("Training encoded final data exported.")

        X_test.to_csv(self.config.testing_independent_data, index=False)
        y_test.to_csv(self.config.testing_dependent_data, index=False)
        logger.info("Exported the test dataset")


        return X_train_resampled, y_train_resampled, X_test, y_test

In [ ]:
try:
    config = ConfigurationManager()
    data_transformer_config = config.get_data_transformation_config()
    data_transformation = DataTransformer(config=data_transformer_config)
    data_transformation.train_test_split()
except Exception as e:
    raise e

[2025-03-23 18:57:07,741: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-23 18:57:07,742: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-23 18:57:07,743: INFO: common: created directory at: artifacts]
[2025-03-23 18:57:07,744: INFO: common: created directory at: artifacts/transformed_data]
[2025-03-23 18:57:07,745: INFO: common: created directory at: artifacts/transformed_data/trainining_data]
[2025-03-23 18:57:07,746: INFO: common: created directory at: artifacts/transformed_data/testing_data]
[2025-03-23 18:57:07,746: INFO: 1553092103: Reading the original dataset for extracting object data type]
[2025-03-23 18:57:07,971: INFO: 1553092103: Extracting columns with object data type]
[2025-03-23 18:57:07,982: INFO: 1553092103: creating a dataframe with columns with no object data type columns]
[2025-03-23 18:57:07,988: INFO: 1553092103: Creating Ordinal encoder object]
[2025-03-23 18:57:08,229: INFO: 1553092103: Encoded the object columns a

In [117]:
%pwd

'/Users/ayushyajnik/Desktop/Python/lapse_rate_app'

In [119]:
df_1 = pd.read_csv("artifacts/transformed_data/trainining_data/X_train.csv")


In [120]:
df_1

,Unnamed: 0,CHANNEL1,CHANNEL2,CHANNEL3,ENTRY AGE,POLICY TYPE 1,POLICY TYPE 2,BENEFIT,SUBSTANDARD RISK,NUMBER OF ADVANCE PREMIUM,INITIAL BENEFIT,Policy Year (Decimal),Policy Year,Premium,SEX,POLICY TYPE 3,PAYMENT MODE,POLICY STATUS,NON LAPSE GUARANTEED
0,0,6.0,2.0,14.000000,26.000000,3.0,9.0,34.000000,0.0,0.0,0.0,2.916667,3.00000,432.0,1.000000,2.0,1.0,3.0,3.0
1,1,2.0,3.0,0.000000,17.000000,3.0,12.0,100.000000,0.0,0.0,0.0,5.916667,6.00000,432.0,0.000000,2.0,1.0,3.0,3.0
2,2,2.0,3.0,0.000000,40.000000,3.0,5.0,5000.000000,0.0,0.0,0.0,3.166667,4.00000,480.0,0.000000,0.0,1.0,3.0,3.0
3,3,6.0,2.0,27.000000,35.000000,15.0,63.0,10000.000000,0.0,0.0,0.0,1.750000,2.00000,120.0,1.000000,0.0,1.0,3.0,3.0
4,4,2.0,3.0,0.000000,44.000000,3.0,16.0,100.000000,0.0,0.0,0.0,1.833333,2.00000,432.0,0.000000,2.0,1.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394360,394360,6.0,2.0,14.000000,22.000000,6.0,30.0,50000.000000,0.0,0.0,0.0,3.750000,4.00000,61.0,0.981384,4.0,2.0,4.0,3.0
394361,394361,2.0,3.0,0.000000,2.612436,3.0,6.0,12000.000000,0.0,0.0,0.0,4.809010,5.00000,480.0,0.000000,3.0,1.0,4.0,3.0
394362,394362,2.0,3.0,0.000000,43.051835,3.0,23.0,6000.000000,0.0,0.0,0.0,1.718922,2.36789,432.0,0.683945,2.0,1.0,4.0,3.0
394363,394363,6.0,2.0,29.128403,30.837871,15.0,63.0,20000.000000,0.0,0.0,0.0,1.319823,2.00000,240.0,1.000000,0.0,1.0,4.0,3.0


In [121]:
df_2 = pd.read_csv("artifacts/transformed_data/trainining_data/y_train.csv")

In [1]:
df_2.shape

NameError: name 'df_2' is not defined

True